In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.losses import Loss
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
def crps(y_true, S):
    """
    Computes continuous ranked probability score:

    Parameters
    ----------
    y_true : tf tensor of shape (BATCH_SIZE, 1, 1)
        True values.
    S : tf tensor of shape (BATCH_SIZE, 1, N_SAMPLES)
        Predictive samples.

    Returns
    -------
    tf tensor of shape (BATCH_SIZE,)
        Scores.

    """
    beta=1
    n_samples = S.shape[-1]
    def expected_dist(diff, beta):
        return K.sum(K.pow(K.sqrt(K.sum(K.square(diff), axis=-2)+K.epsilon()), beta),axis=-1)
    es_1 = expected_dist(y_true - S, beta)
    es_2 = 0
    for i in range(n_samples):
        es_2 = es_2 + expected_dist(K.expand_dims(S[:,:,i]) - S, beta)
    return es_1/n_samples - es_2/(2*n_samples**2)

class CRPSLoss(Loss):
    def call(self, y_true, S):
        return crps(y_true, S)

In [3]:
import CRPS.CRPS as pscore

y_pred = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_true = [20.0]

# CRPS normal berechnen
crps_normal = pscore(np.array(y_pred), y_true).compute()[0]
print(crps_normal)

# Umwandeln der Werte in TensorFlow-Tensoren und Anpassung der Dimensionen
y_pred_tensor = tf.expand_dims(tf.constant(y_pred, dtype=tf.float32), axis=0)  # Fügt die Batch-Dimension hinzu
y_true_tensor = tf.expand_dims(tf.constant(y_true, dtype=tf.float32), axis=-1)  # Fügt die Dimension (D, 1) hinzu

# Aufrufen der CRPS-Funktion
result = crps(y_true_tensor, y_pred_tensor)

print("CRPS-Score:", result.numpy())  # Ergebnis ausgeben

[12.85]


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Index out of range using input dim 2; input has only 2 dims [Op:StridedSlice] name: strided_slice/